# Pauli Propagation: Theory Tutorial

This notebook introduces **Pauli propagation**, the core algorithm behind `pprop`. We work through a small example by hand and then verify every step with the library.

## Background

Computing the expectation value of an observable $O$ under a parametrized circuit $U(\boldsymbol{\theta})$ starting from $|0\rangle$ amounts to evaluating

$$\langle O \rangle(\boldsymbol{\theta}) = \langle 0 | U^\dagger(\boldsymbol{\theta})\, O\, U(\boldsymbol{\theta}) | 0 \rangle.$$

The standard approach simulates the statevector $U(\boldsymbol{\theta})|0\rangle$, which costs $O(2^n)$ memory. Pauli propagation takes the opposite route: it evolves $O$ **backwards** through the circuit in the Heisenberg picture,

$$O \;\to\; U^\dagger\, O\, U,$$

keeping track of the result as a **sum of Pauli words** with trigonometric polynomial coefficients. At the end, only the Pauli words made of $I$ or $Z$ contribute to the final expectation value.

In the $|0\rangle^{\otimes n}$ computational basis state, only Pauli words composed entirely of $Z$ and $I$ operators have non-zero expectation:

$$
        \langle 0 | I | 0 \rangle = 1, \quad
        \langle 0 | Z | 0 \rangle = 1, \quad
        \langle 0 | X | 0 \rangle = 0, \quad
        \langle 0 | Y | 0 \rangle = 0
$$

## 1. Propagation rules

Each gate $G(\theta)$ defines a **conjugation map** on Pauli operators $P \mapsto G^\dagger(\theta)\, P\, G(\theta)$. For the two gates in our example:

**RY gate** ($RY(\theta) = e^{-i\theta Y/2}$):

$$X \to \cos\theta\, X + \sin\theta\, Z$$
$$Z \to \cos\theta\, Z - \sin\theta\, X$$

**CNOT gate** (control on qubit 1, target on qubit 0):

$$Z \otimes I \to Z \otimes Z$$
$$X \otimes I \to X \otimes I$$

All other single-qubit Paulis on qubits not in the gate support are left unchanged. The rules above are derived directly from the conjugation relations and hold exactly for any parameter value.

## 2. The circuit

We propagate the observable $Z_0$ (Pauli Z on qubit 0) backwards through the following two-qubit circuit:

```
q0: RY(theta_0) -- [target of CNOT] -- RY(theta_1) -- measure Z
q1:              -- [control of CNOT]
```

Note the gate order for propagation is **reversed**: we start from the observable and apply gates from right to left, so the sequence is $RY(\theta_1)$, then CNOT, then $RY(\theta_0)$.

In [1]:
from pprop import Propagator
import pennylane as qml

def ansatz(params):
    qml.RY(params[0], wires=0)
    qml.CNOT([1, 0])
    qml.RY(params[1], wires=0)
    return qml.expval(qml.Z(0))

prop = Propagator(ansatz)
prop.show()

0: ──RY(0.00)─╭X──RY(1.00)─┤  <Z>
1: ───────────╰●───────────┤     


/home/samonaco/Pauli-Propagator/.venv/lib/python3.12/site-packages/pennylane/operation.py:2622: PennyLaneDeprecationWarning: Observable is deprecated and will be removed in v0.43. A generic Operator class should be used instead. If defining an Operator, set the is_hermitian property to True. If checking if an Operator is Hermitian, check the is_hermitian property. 
  warnings.warn(


## 3. Propagation by hand

We track the evolving Pauli word step by step. Gates are applied in reverse circuit order.

**Starting observable:** $Z_0 \otimes I_1$

---

**Step 1:** Apply $RY(\theta_1) \otimes I$ using the rule $Z \to \cos\theta Z - \sin\theta X$:

$$Z_0 \otimes I_1 \;\to\; \cos(\theta_1)\, Z_0 \otimes I_1 \;-\; \sin(\theta_1)\, X_0 \otimes I_1$$

---

**Step 2:** Apply CNOT(1, 0) using the rule $Z_0 \otimes I_1 \to Z_0 \otimes Z_1$ (X on the target is unchanged):

$$\cos(\theta_1)\, Z_0 \otimes I_1 - \sin(\theta_1)\, X_0 \otimes I_1
\;\to\;
\cos(\theta_1)\, Z_0 \otimes Z_1 - \sin(\theta_1)\, X_0 \otimes I_1$$

---

**Step 3:** Apply $RY(\theta_0) \otimes I$ using $Z \to \cos\theta Z - \sin\theta X$ and $X \to \cos\theta X + \sin\theta Z$:

$$\cos(\theta_1)\, Z_0 \otimes Z_1 - \sin(\theta_1)\, X_0 \otimes I_1$$

$$\to\; +\cos(\theta_0)\cos(\theta_1)\, Z_0 \otimes Z_1
\;-\; \sin(\theta_0)\cos(\theta_1)\, X_0 \otimes Z_1$$

$$\;-\; \cos(\theta_0)\sin(\theta_1)\, X_0 \otimes I_1
\;-\; \sin(\theta_0)\sin(\theta_1)\, Z_0 \otimes I_1$$

---

**Step 4 (trimming):** $\langle 0|\cdot|0\rangle$ is non-zero only for the identity on every qubit. Among the four terms above, only $Z_0 \otimes I_1$ contributes (since $\langle 0|Z|0\rangle = 1$ and $\langle 0|I|0\rangle = 1$, while $\langle 0|X|0\rangle = 0$). Therefore:

$$\langle Z_0 \rangle(\boldsymbol{\theta}) = -\sin(\theta_0)\sin(\theta_1) + \cos(\theta_0)\cos(\theta_1)$$

## 4. Verify with pprop: debug mode

Calling `prop.propagate(debug=True)` prints each gate application, showing the Pauli dict before and after.

In [2]:
prop.propagate(debug=True)

Propagating (1.00)*Z0
=== Evolve ===
GATE: RY(1, Wires([0]))
 PRE: (1.00)*Z0
  REM: ()*Z0
  ADD: (1.00*cos(θ_1))*Z0 + (-1.00*sin(θ_1))*X0
POST: (1.00*cos(θ_1))*Z0 + (-1.00*sin(θ_1))*X0
=== Evolve ===
GATE: CNOT(Wires([1, 0]))
 PRE: (1.00*cos(θ_1))*Z0 + (-1.00*sin(θ_1))*X0
  REM: ()*Z0 + ()*X0
  ADD: (1.00*cos(θ_1))*Z0 Z1 + (-1.00*sin(θ_1))*X0
POST: (1.00*cos(θ_1))*Z0 Z1 + (-1.00*sin(θ_1))*X0
=== Evolve ===
GATE: RY(0, Wires([0]))
 PRE: (1.00*cos(θ_1))*Z0 Z1 + (-1.00*sin(θ_1))*X0
  REM: ()*Z0 Z1 + ()*X0
  ADD: (1.00*cos(θ_1)*cos(θ_0))*Z0 Z1 + (-1.00*sin(θ_0)*cos(θ_1))*X0 Z1 + (-1.00*sin(θ_1)*cos(θ_0))*X0 + (-1.00*sin(θ_1)*sin(θ_0))*Z0
POST: (1.00*cos(θ_1)*cos(θ_0))*Z0 Z1 + (-1.00*sin(θ_0)*cos(θ_1))*X0 Z1 + (-1.00*sin(θ_1)*cos(θ_0))*X0 + (-1.00*sin(θ_1)*sin(θ_0))*Z0


## 5. The closed-form expression

`prop.expression(0)` returns the symbolic trigonometric polynomial for the first (and only) observable. It should match the hand-derived result.

In [3]:
prop.expression(0)

-1.0*sin(θ0)*sin(θ1) + 1.0*cos(θ0)*cos(θ1)

## 6. Numerical evaluation

We can now evaluate $\langle Z_0 \rangle$ at any parameter point in microseconds, with no statevector simulation. We verify against the analytical formula.

In [11]:
import numpy as np

rng    = np.random.default_rng(0)
params = rng.uniform(0, 2 * np.pi, size=prop.num_params)

pprop_val    = prop(params)[0]
analytic_val = -np.sin(params[0])*np.sin(params[1]) + np.cos(params[0])*np.cos(params[1])

print(f"pprop value    : {pprop_val:.10f}")
print(f"Analytic value : {analytic_val:.10f}")
print(f"Difference     : {abs(pprop_val - analytic_val):.2e}")

pprop value    : 0.8332053265
Analytic value : 0.8332053265
Difference     : 0.00e+00


## 7. Exact gradients

Because the expression is a trigonometric polynomial, its gradient with respect to $\boldsymbol{\theta}$ is also exact. `eval_and_grad` returns both the value and the gradient vector in a single call.

The analytical gradient are:

$$\frac{\partial}{\partial \theta_0} = -\cos(\theta_0)\sin(\theta_1) - \sin(\theta_0)\cos(\theta_1), \qquad \frac{\partial}{\partial \theta_1} = -\sin(\theta_0)\cos(\theta_1) - \cos(\theta_0)\sin(\theta_1)$$

In [13]:
vals, grads = prop.eval_and_grad(params)

pprop_grad    = np.array(grads[0])
analytic_grad = np.array([
    -np.cos(params[0])*np.sin(params[1]) - np.sin(params[0])*np.cos(params[1]),
    -np.sin(params[0])*np.cos(params[1]) - np.cos(params[0])*np.sin(params[1]),
])

print(f"pprop gradient    : {pprop_grad}")
print(f"Analytic gradient : {analytic_grad}")
print(f"Max difference    : {np.max(np.abs(pprop_grad - analytic_grad)):.2e}")

pprop gradient    : [0.55296373 0.55296373]
Analytic gradient : [0.55296373 0.55296373]
Max difference    : 0.00e+00


## 8. Truncation

For larger circuits, exact propagation is intractable because the number of Pauli words grows exponentially with depth. `pprop` handles this with two cutoffs:

- `k1`: maximum Pauli weight (number of non-identity single-qubit factors). Words with weight above `k1` are discarded.
- `k2`: maximum trigonometric degree (total number of sin/cos factors in a term's coefficient). Terms above `k2` are discarded.

On this two-qubit example both cutoffs are loose enough that the result is exact. We verify this by comparing truncated and exact propagation at the same parameter point.

In [6]:
def ansatz_6(params):
    num_qubits = 6
    param_idx = 0
    for qubit in range(num_qubits):
        qml.RY(params[param_idx], wires=qubit)
        param_idx += 1

    qml.Barrier()

    for qubit in range(0, num_qubits-1, 2):
        qml.CNOT([qubit, qubit+1])
    for qubit in range(1, num_qubits-1, 2):
        qml.CNOT([qubit, qubit+1])

    qml.Barrier()

    for qubit in range(num_qubits):
        qml.RX(params[param_idx], wires=qubit)
        param_idx += 1

    qml.Barrier()
        
    for qubit in range(0, num_qubits-1, 2):
        qml.CNOT([qubit, qubit+1])
    for qubit in range(1, num_qubits-1, 2):
        qml.CNOT([qubit, qubit+1])

    qml.Barrier()

    for qubit in range(num_qubits):
        qml.RY(params[param_idx], wires=qubit)
        param_idx += 1

    return qml.expval(qml.Z(0))

In [14]:
prop_exact     = Propagator(ansatz_6)
prop_truncated = Propagator(ansatz_6, k1=3, k2=5)

prop_exact.propagate(opt=True)
prop_truncated.propagate(opt=True)

In [15]:
prop_exact.show()

0: ──RY(0.00)──||─╭●─────||──RX(6.00)───||─╭●─────||──RY(12.00)─┤  <Z>
1: ──RY(1.00)──||─╰X─╭●──||──RX(7.00)───||─╰X─╭●──||──RY(13.00)─┤     
2: ──RY(2.00)──||─╭●─╰X──||──RX(8.00)───||─╭●─╰X──||──RY(14.00)─┤     
3: ──RY(3.00)──||─╰X─╭●──||──RX(9.00)───||─╰X─╭●──||──RY(15.00)─┤     
4: ──RY(4.00)──||─╭●─╰X──||──RX(10.00)──||─╭●─╰X──||──RY(16.00)─┤     
5: ──RY(5.00)──||─╰X─────||──RX(11.00)──||─╰X─────||──RY(17.00)─┤     


In [16]:
rng    = np.random.default_rng(0)
params = rng.uniform(0, 2 * np.pi, size=prop_exact.num_params)

val_exact     = prop_exact(params)[0]
val_truncated = prop_truncated(params)[0]

print(f"Exact value     : {val_exact:.10f}")
print(f"Truncated value : {val_truncated:.10f}")
print(f"Difference      : {abs(val_exact - val_truncated):.2e}")

Exact value     : 0.3036714265
Truncated value : 0.3192962423
Difference      : 1.56e-02
